In [1]:
import numpy as np

In [2]:
import numpy as np
def _tridiagonal_solve(a,b,c,d):
    n=len(d)
    cp=np.empty(n-1)
    dp=np.empty(n)
    cp[0]=c[0]/b[0]
    dp[0]=d[0]/b[0]
    for i in range(1,n-1):
        denom=b[i]-a[i-1]*cp[i-1]
        cp[i]=c[i]/denom
        dp[i]=(d[i]-a[i-1]*dp[i-1])/denom
    dp[n-1]=(d[n-1]-a[n-2]*dp[n-2])/(b[n-1]-a[n-2]*cp[n-2])
    x=np.empty(n)
    x[n-1]=dp[n-1]
    for i in range(n-2,-1,-1):
        x[i]=dp[i]-cp[i]*x[i+1]
    return x
def _domain_from_L(K,sigma,T,L):
    xmin=np.log(K)-L*sigma*np.sqrt(T)
    xmax=np.log(K)+L*sigma*np.sqrt(T)
    return xmin,xmax
def _boundaries(is_call,american,K,r,q,Smin,Smax,tau):
    if is_call:
        left=0.0
        right=np.exp(-q*tau)*Smax-K*np.exp(-r*tau)
    else:
        left=K if american else K*np.exp(-r*tau)
        right=0.0
    return left,right
def solve_pde_explicit(K,T,r,sigma,q=0.0,is_call=False,american=True,M=200,N=5000,L=6.0,xmin=None,xmax=None,return_grid=False):
    if xmin is None or xmax is None:
        xmin,xmax=_domain_from_L(K,sigma,T,L)
    x=np.linspace(xmin,xmax,M+1)
    S=np.exp(x)
    dx=(xmax-xmin)/M
    dt=T/N
    a=0.5*sigma*sigma
    b=r-q-0.5*sigma*sigma
    alpha=a*dt/(dx*dx)
    beta=b*dt/(2.0*dx)
    gamma=r*dt
    payoff=np.maximum(S-K,0.0) if is_call else np.maximum(K-S,0.0)
    U=np.empty((N+1,M+1))
    U[0]=payoff
    for n in range(N):
        tau=(n+1)*dt
        left,right=_boundaries(is_call,american,K,r,q,np.exp(xmin),np.exp(xmax),tau)
        u=U[n].copy()
        u[0]=left
        u[-1]=right
        un=u.copy()
        un[1:-1]=u[1:-1]+alpha*(u[2:]-2.0*u[1:-1]+u[:-2])+beta*(u[2:]-u[:-2])-gamma*u[1:-1]
        if american:
            un=np.maximum(un,payoff)
        U[n+1]=un
    if return_grid:
        return S,U[-1],U
    return S,U[-1]
def solve_pde_implicit(K,T,r,sigma,q=0.0,is_call=False,american=True,M=200,N=1000,L=6.0,xmin=None,xmax=None,return_grid=False):
    if xmin is None or xmax is None:
        xmin,xmax=_domain_from_L(K,sigma,T,L)
    x=np.linspace(xmin,xmax,M+1)
    S=np.exp(x)
    dx=(xmax-xmin)/M
    dt=T/N
    a=0.5*sigma*sigma
    b=r-q-0.5*sigma*sigma
    alpha=a*dt/(dx*dx)
    beta=b*dt/(2.0*dx)
    gamma=r*dt
    payoff=np.maximum(S-K,0.0) if is_call else np.maximum(K-S,0.0)
    U=np.empty((N+1,M+1))
    U[0]=payoff
    l=np.full(M-1,-alpha-beta)
    d=np.full(M-1,1.0+2.0*alpha+gamma)
    u=np.full(M-1,-alpha+beta)
    for n in range(N):
        tau=(n+1)*dt
        left,right=_boundaries(is_call,american,K,r,q,np.exp(xmin),np.exp(xmax),tau)
        rhs=U[n,1:-1].copy()
        rhs[0]+= (alpha+beta)*left
        rhs[-1]+= (alpha-beta)*right
        y=_tridiagonal_solve(l.copy(),d.copy(),u.copy(),rhs)
        un=U[n].copy()
        un[0]=left
        un[-1]=right
        un[1:-1]=y
        if american:
            un=np.maximum(un,payoff)
        U[n+1]=un
    if return_grid:
        return S,U[-1],U
    return S,U[-1]